# Load run data and convert to pandas dataframe

In [ ]:
# %matplotlib notebook

from wild_visual_navigation import WVN_ROOT_DIR
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

folder, model_name = "confidence_fn", False
folder, model_name = "lr", False
folder, model_name = "feature", False
folder, model_name = "loss", False
folder, model_name = "network", False


folder, model_name = "threshold", False

folder, model_name = "anomaly_detection_only", False
folder, model_name = "classicial_learning", True
# folder, model_name = "debug", False
# p = os.path.join(WVN_ROOT_DIR, f"scripts/ablations/{folder}_ablation/{folder}_ablation_test_results.pkl")
ws = "_" + os.environ["ENV_WORKSTATION_NAME"]
p = os.path.join(WVN_ROOT_DIR, f"results/ablations/{folder}_ablation{ws}/{folder}_ablation_test_results.pkl")
with open(p, "rb") as f:
    res = pickle.load(f)

import scipy
import pandas as pd
import pickle
import copy

data = {
    "run_nr": [],
    "cfg_name": [],
    "train_scene": [],
    "test_scene": [],
}

for train_scene_name, scene_data in res.items():
    for config_path, config_data in scene_data.items():
        if model_name:
            name = config_path
        else:
            # its a path
            name = config_path.split("/")[-1][:-5]
        for run_nr, run_data in config_data.items():
            for test_scene_name, test_scene_data in run_data.items():
                data["cfg_name"].append(name)
                data["run_nr"].append(run_nr)
                data["train_scene"].append(train_scene_name)
                data["test_scene"].append(test_scene_name)
                for k in test_scene_data.keys():
                    if type(test_scene_data[k]) is float:
                        try:
                            data[k].append(test_scene_data[k])
                        except:
                            data[k] = [test_scene_data[k]]

df = pd.DataFrame.from_dict(data)

data_acc = {
    "cfg_name": [],
    "train_scene": [],
    "test_scene": [],
}

data_auroc = {
    "cfg_name": [],
    "train_scene": [],
    "test_scene": [],
}
keys = []
for i in range(len(df)):
    n = data["cfg_name"][i]
    tr = data["train_scene"][i]
    te = data["test_scene"][i]

    a = df[(df["train_scene"] == tr) * (df["test_scene"] == te) * (df["cfg_name"] == n)]
    k = n + tr + te
    if k in keys:
        continue
    keys.append(k)

    data_auroc["cfg_name"].append(n)
    data_auroc["train_scene"].append(tr)
    data_auroc["test_scene"].append(te)

    data_acc["cfg_name"].append(n)
    data_acc["train_scene"].append(tr)
    data_acc["test_scene"].append(te)

    for k in a.keys():

        if k.find("auroc") != -1:
            try:
                data_auroc[k + "_std"].append(a[k].std())
                data_auroc[k + "_m"].append(a[k].mean())
            except:
                data_auroc[k + "_std"] = [a[k].std()]
                data_auroc[k + "_m"] = [a[k].mean()]
        if k.find("acc") != -1:
            try:
                data_acc[k + "_std"].append(a[k].std())
                data_acc[k + "_m"].append(a[k].mean())
            except:
                data_acc[k + "_std"] = [a[k].std()]
                data_acc[k + "_m"] = [a[k].mean()]
df_auroc = pd.DataFrame.from_dict(data_auroc)
df_auroc = df_auroc.round(decimals=4)
df_auroc[df_auroc.select_dtypes(include=["number"]).columns] = (
    df_auroc[df_auroc.select_dtypes(include=["number"]).columns] * 100
)

df_acc = pd.DataFrame.from_dict(data_acc)
df_acc = df_acc.round(decimals=4)
df_acc[df_acc.select_dtypes(include=["number"]).columns] = (
    df_acc[df_acc.select_dtypes(include=["number"]).columns] * 100
)

In [ ]:
df_acc = df_acc.sort_values("test_acc_gt_image_m", ascending=False)
scene = "hilly"
df_acc[(df_acc["train_scene"].str.find(scene) != -1)]  # * (df_acc["test_scene"].str.find(scene) != -1)

In [ ]:
df_acc = df_acc.sort_values("test_acc_gt_image_m", ascending=False)
scene = "forest"
df_acc[(df_acc["train_scene"].str.find(scene) != -1)]  # (df["test_scene"].str.find(scene) != -1)

In [ ]:
df_acc = df_acc.sort_values("test_acc_gt_image_m", ascending=False)
scene = "grassland"
df_acc[(df_acc["train_scene"].str.find(scene) != -1)]  # (df["test_scene"].str.find(scene) != -1)